## Hyper Parameter Tuning using GridSearchCV(Cross-Validation)

### Importing libraries

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

### Importing dataset

In [2]:
data = pd.read_csv('dataset/Churn_Modelling.csv')
X = data.iloc[:, 3:13] # Drop the three first features
y = data.iloc[:, 13]

### OneHotEncoding categorical features

In [3]:
cat_cols = [col_name for col_name in X.columns if X[col_name].nunique() < 10 and X[col_name].dtype == "object"]
for col_name in cat_cols:
    dummy_col = pd.get_dummies(X[col_name])
    X = pd.concat([X, dummy_col], axis=1)
    # drop categorical column
    X = X.drop([col_name], axis=1)

### Spliting training(ratio=0.8) and testing set 


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature scaling 

In [5]:
from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()
X_train = std_scale.fit_transform(X_train)
X_test = std_scale.fit_transform(X_test)

## Performing Hyper-parameter Optimization 

## Importing libraries

In [21]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid

### Create NN model 

In [28]:
def create_nn_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i == 0:   # architecture of the input layer / should provide the no. of imput neurons 
            model.add(Dense(nodes, input_dim=X_train.shape[1]))
        else:
            model.add(Dense(nodes))
        # Activation functions and Dropout ratio 
        model.add(Activation(activation))
        model.add(Dropout(0.3))
        
    # architecture of the output layer
    model.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
    # Setup optimizer: loss function(binary_crossentropy=for binary output); loss metrics
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

### Initializing the model 

In [29]:
model = KerasClassifier(build_fn=create_nn_model, verbose=0)

### Setup trials parameters for GridSearchCV optimization 

In [30]:
layers = [[20], [40,20], [45,30,15]]
activations = ['sigmoid', 'relu']
grid_params = dict(layers=layers, activation=activations, batch_size=[128,256], epochs=[30])

### Initializing GridSearchCV - making 'model' as estimator model  

In [31]:
grid_estimator = GridSearchCV(estimator=model, param_grid=grid_params, cv=5)

### Fit the model with the optimized parameters from 'grid_estimator'


In [32]:
grid_result = grid_estimator.fit(X_train, y_train)

Epoch 1/30
6400/6400 [==============================] - 4s 594us/step - loss: 0.5628 - acc: 0.7411
Epoch 2/30
6400/6400 [==============================] - 0s 76us/step - loss: 0.4997 - acc: 0.7900
Epoch 3/30
6400/6400 [==============================] - 1s 95us/step - loss: 0.4852 - acc: 0.7945
Epoch 4/30
6400/6400 [==============================] - 0s 76us/step - loss: 0.4698 - acc: 0.7987
Epoch 5/30
6400/6400 [==============================] - 1s 85us/step - loss: 0.4657 - acc: 0.7992
Epoch 6/30
6400/6400 [==============================] - 0s 70us/step - loss: 0.4626 - acc: 0.7983
Epoch 7/30
6400/6400 [==============================] - 0s 59us/step - loss: 0.4583 - acc: 0.8013
Epoch 8/30
6400/6400 [==============================] - 0s 78us/step - loss: 0.4507 - acc: 0.8033
Epoch 9/30
6400/6400 [==============================] - 0s 74us/step - loss: 0.4524 - acc: 0.8030
Epoch 10/30
6400/6400 [==============================] - 1s 94us/step - loss: 0.4491 - acc: 0.8023
Epoch 11/30
6400/6

### Show the best params 

In [33]:
print(grid_result.best_score_, grid_result.best_params_)

0.85575 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [45, 30, 15]}


### Confusion Matrix 

In [3]:
y_predict = model.predict(X_test)
y_predict = (y_predict > 0.5)
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predict)
print(cm)

NameError: name 'model' is not defined